# RNN

-입력과 출력을 시퀀스로 처리하는 모델
-은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있음

![image.png](attachment:image.png)


  일 대 다 : 하나의 입력에 대해서 여러개의 출력(one-to-many)
• 하나의 사진 이미지 입력에 대해서 사진의 제목을 출력을 내놓는 이미지 캡셔닝(Image
Captioning) 작업에 사용
• 사진의 제목은 단어들의 나열이므로 시퀀스의 형태를 가짐
 다 대 일 : 다수의 입력에 대해서 하나의 출력(many-to-one)
• 입력 데이터으로부터 긍정적 감성인지 부정적 감성인지를 판별하는 감성 분류
(Sentiment Classification),
• 입력 데이터가 어떤 종류의 문서인지를 판별하는 문서 분류(Document Classification)에
사용
 다 대 다(many-to-many)
• 입력 문장으로 부터 대답을 출력하는 챗봇, 입력 문장으로부터 번역된 문장을 출력하
는 번역기, 개체명 인식, 품사 태깅 등에 이용


# RNN 실습
![image.png](attachment:image.png)





hidden_size = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.
timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.
input_dim = 입력의 크기.


In [0]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()



Using TensorFlow backend.
W0821 02:40:24.065211 140576597014400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 02:40:24.104816 140576597014400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 02:40:24.114213 140576597014400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


출력값이 (batch_size, output_dim) 크기의 2D 텐서일 때, output_dim은 hidden_size의 값인 3입니다. 이 경우 batch_size를 현 단계에서는 알 수 없으므로 (None, 3)이 됨


In [0]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


batch_size를 8로 기재하자, 출력의 크기가 (8, 3)이 된 것을 볼 수 있습니다. 이제 return_sequences 매개 변수에 True를 기재하여 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하도록 모델을 만들어 보자

In [0]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


# RNN 이용한 언어 모델링 

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import SimpleRNN
tf.enable_eager_execution()

In [0]:
train_X=[[0.1,4.2,1.5,1.1,2.8],[1.0,3.1,2.5,0.7,1.1],[
    .3,2.1,1.5,2.1,0.1],[2.2,1.4,0.5,0.9,1.1]]

print(np.shape(train_X))


(4, 5)


 영웅은', '죽지', '않아요', '너만'이라는 단어들을 각 시점의 훈련 데이터인 X로
사용
 여기서는 RNN의 입력으로 임베딩 벡터(embedding vector)를 사용
이는 2차원 텐서에 해당되며 (input_length, input_dim)의 크기에 해당됨

In [0]:
train_X=[[[0.1,4.2,1.5,1.1,2.8],[1.0,3.1,2.5,0.7,1.1],[
    .3,2.1,1.5,2.1,0.1],[2.2,1.4,0.5,0.9,1.1]]]

train_X=np.array(train_X,dtype=np.float32)

print(train_X.shape)

(1, 4, 5)


이를 RNN에 넣어서 RNN이 어떤 결과를 리턴하는지 확인
 RNN은 2D 텐서가 아니라 3D 텐서를 입력 받기 때문에
위에서 만든 2D 텐서를 3D 텐서로 변경
 (batch_size, timesteps, input_dim)에 해당되는 (1, 4, 5)의 크기를 가지는 3D 텐서
생성
 batch_size는 한 번에 RNN이 학습하는 데이터의 양을 설정하는데,
여기서는 사실 학습하는 데이터 즉, 문장이 1개 밖에 없으므로
batch_size는 1

In [0]:
rnn=SimpleRNN(3,return_sequences=True,return_state=True)

hidden_states,last_states=rnn(train_X)

print('train_X :{}, shape: {}'.format(train_X,train_X.shape))   #입력으로 사용한 훈련 데이터

print('hidden states : {} , shape: {}'.format(hidden_states,hidden_states.shape))   #모든 time-step의 은닉상태

print('last hidden state : {}, shape: {}'.format(last_states,last_states.shape))   #마지막 은닉 상태

train_X :[[[0.1 4.2 1.5 1.1 2.8]
  [1.  3.1 2.5 0.7 1.1]
  [0.3 2.1 1.5 2.1 0.1]
  [2.2 1.4 0.5 0.9 1.1]]], shape: (1, 4, 5)
hidden states : [[[-0.91957676 -0.9997508  -0.05448371]
  [-0.42947188 -0.99982375  0.9265283 ]
  [-0.93827075 -0.90613776  0.38662696]
  [-0.87534666 -0.9901501   0.49548334]]] , shape: (1, 4, 3)
last hidden state : [[-0.87534666 -0.9901501   0.49548334]], shape: (1, 3)


출력층에 대한 설계

앞서 은닉층의 RNN 셀이 각각의 시점에 대해서 은닉 상태 벡터의 크기
즉, output_dim이 3인 벡터를 만들어내는 것을 확인
• 그런데 실제값 y인 원-핫 벡터는 차원이 5이므로, 예측값 또한 출력층을 통해
output_dim을 다시 5로 바꿔줄 필요가 있음. 그래야 손실 함수를 적용할 수 있기 때문
• 이는 출력층에 model.add(Dense(5))를 통해 5개의 뉴런을 추가하면 해결

다음 단어 예측

In [0]:
from keras_preprocessing.text import Tokenizer

text="나랑 점심 먹으로 갈래 메뉴는 햄버거 점심 메뉴 좋지"

t=Tokenizer()
t.fit_on_texts([text])
encoded=t.texts_to_sequences([text])[0]

#[0]을 해주지 않으면 [[contents]]와 같은 리스트 안의 리스트 형태로 저장 됨
     # 해주면  [contents] 와 같은 리스트로 저장
    



In [0]:
vocab_size=len(t.word_index)+1
#케라스 토크나이저의 정수 인코딩은 인데스가 1부터 시작하지만 원핫 인코딩에서 배열의 인덱스가 0부터시작하기 때문에 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성

print(t.word_index)

{'점심': 1, '나랑': 2, '먹으로': 3, '갈래': 4, '메뉴는': 5, '햄버거': 6, '메뉴': 7, '좋지': 8}


문장에서 두 개 단어씩 묶기 

In [0]:
sequences=list()
for c in range(1,len(encoded)):
    sequence=encoded[c-1 : c+1]   #단어를 두개씩 묶어서 저장 이는 x,의 관계                                      구성하기위해
    sequences.append(sequence)
    
print('단어 묶음의 개수 : %d '%len(sequences))

단어 묶음의 개수 : 8 


In [0]:
print(sequences)

[[2, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 1], [1, 7], [7, 8]]


해당 단어의 묶음에서 첫번째 열을 X, 두번째 열을 y로 저장한다면,
X를 현재 등장한 단어, y를 다음에 등장할 단어로 하여
훈련 데이터로 사용할 수 있음

In [0]:
import numpy as np
X,y=zip(*sequences) #첫 번째 열이 X, 두번째 열이 y

X=np.array(X)
y=np.array(y)  



y 에 대해 원핫 인코딩 

In [0]:

from keras.utils import to_categorical
y=to_categorical(y,num_classes=vocab_size)  #원핫 인코딩

print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [0]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential
tf.compat.v1.disable_eager_execution()

model=Sequential()
model.add(Embedding(vocab_size,9,input_length=1))
#단어 집합의 크기는9 임베딩 벡터의 크기는 9 각 샘플 길이는 단어 한 개이므로 길이는1

model.add(SimpleRNN(9))
#RNN의 결과값으로 나오는 벡터의 차원 9로 함

model.add(Dense(vocab_size,activation='softmax'))
#출력층을 지나서 나오는 벡터의 크기도 9로 함



W0821 02:45:45.114322 139779097495424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 02:45:45.117377 139779097495424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 02:45:45.122581 139779097495424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=500,verbose=2)

W0821 02:47:18.696825 139779097495424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0821 02:47:18.727393 139779097495424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0821 02:47:19.001504 139779097495424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0821 02:47:19.195265 139779097495424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Epoch 1/500
 - 1s - loss: 2.1934 - acc: 0.1250
Epoch 2/500
 - 0s - loss: 2.1905 - acc: 0.1250
Epoch 3/500
 - 0s - loss: 2.1873 - acc: 0.1250
Epoch 4/500
 - 0s - loss: 2.1838 - acc: 0.1250
Epoch 5/500
 - 0s - loss: 2.1803 - acc: 0.1250
Epoch 6/500
 - 0s - loss: 2.1767 - acc: 0.1250
Epoch 7/500
 - 0s - loss: 2.1730 - acc: 0.2500
Epoch 8/500
 - 0s - loss: 2.1694 - acc: 0.3750
Epoch 9/500
 - 0s - loss: 2.1656 - acc: 0.3750
Epoch 10/500
 - 0s - loss: 2.1619 - acc: 0.3750
Epoch 11/500
 - 0s - loss: 2.1581 - acc: 0.5000
Epoch 12/500
 - 0s - loss: 2.1544 - acc: 0.5000
Epoch 13/500
 - 0s - loss: 2.1506 - acc: 0.5000
Epoch 14/500
 - 0s - loss: 2.1468 - acc: 0.6250
Epoch 15/500
 - 0s - loss: 2.1430 - acc: 0.6250
Epoch 16/500
 - 0s - loss: 2.1392 - acc: 0.7500
Epoch 17/500
 - 0s - loss: 2.1353 - acc: 0.7500
Epoch 18/500
 - 0s - loss: 2.1314 - acc: 0.8750
Epoch 19/500
 - 0s - loss: 2.1275 - acc: 0.8750
Epoch 20/500
 - 0s - loss: 2.1236 - acc: 0.8750
Epoch 21/500
 - 0s - loss: 2.1196 - acc: 0.8750
E

**입력한 단어에 대해 다음 단어를 정상적으로 예측하는지 확인**

In [0]:
print(t.word_index.items())

dict_items([('점심', 1), ('나랑', 2), ('먹으로', 3), ('갈래', 4), ('메뉴는', 5), ('햄버거', 6), ('메뉴', 7), ('좋지', 8)])




> 다음 단어를 출력하는 함수




In [0]:
def predict_next_word(model, t, current_word):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = np.array(encoded)
    result = model.predict_classes(encoded, verbose=0)
    for word, index in t.word_index.items():
        if index == result:
            return word

주어진 단어로 부터 문장을 생성하는 함수

In [0]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ""
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = np.array(encoded)
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = word
        sentence = sentence + " " + word
    
    sentence = init_word + sentence
    return sentence

# **문맥을 반영해서 다음 단어 예측**

In [0]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [0]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [0]:
encoded

[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]

# 토큰화와 정수 인코딩 과정

In [0]:
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [0]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


# 훈련 데이터

In [0]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]

In [0]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [0]:
print(max(len(l) for l in sequences)) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력

6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인하였습니다. 이제 전체 샘플의 길이를 6으로 패딩합니다.

In [0]:
from keras.preprocessing.sequence import pad_sequences
max_len=6
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며, padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채움

In [0]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


각 샘플의 마지막 단어를 레이블로 분리

In [0]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [0]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [0]:
print(y) # 모든 샘플에 대한 레이블 출력

[ 3  1  4  5  1  7  1  9 10  1 11]


이제 RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행

In [0]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [0]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# 훈련

In [0]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 2.4510 - acc: 0.2727
Epoch 2/200
 - 0s - loss: 2.4375 - acc: 0.3636
Epoch 3/200
 - 0s - loss: 2.4237 - acc: 0.5455
Epoch 4/200
 - 0s - loss: 2.4095 - acc: 0.5455
Epoch 5/200
 - 0s - loss: 2.3948 - acc: 0.5455
Epoch 6/200
 - 0s - loss: 2.3795 - acc: 0.4545
Epoch 7/200
 - 0s - loss: 2.3635 - acc: 0.4545
Epoch 8/200
 - 0s - loss: 2.3469 - acc: 0.4545
Epoch 9/200
 - 0s - loss: 2.3295 - acc: 0.4545
Epoch 10/200
 - 0s - loss: 2.3113 - acc: 0.3636
Epoch 11/200
 - 0s - loss: 2.2922 - acc: 0.3636
Epoch 12/200
 - 0s - loss: 2.2721 - acc: 0.3636
Epoch 13/200
 - 0s - loss: 2.2510 - acc: 0.3636
Epoch 14/200
 - 0s - loss: 2.2290 - acc: 0.3636
Epoch 15/200
 - 0s - loss: 2.2059 - acc: 0.3636
Epoch 16/200
 - 0s - loss: 2.1820 - acc: 0.3636
Epoch 17/200
 - 0s - loss: 2.1572 - acc: 0.3636
Epoch 18/200
 - 0s - loss: 2.1318 - acc: 0.3636
Epoch 19/200
 - 0s - loss: 2.1058 - acc: 0.3636
Epoch 20/200
 - 0s - loss: 2.0795 - acc: 0.3636
Epoch 21/200
 - 0s - loss: 2.0532 - acc: 0.3636
E

문장을 생성하는 함수

In [0]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [0]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

경마장에 있는 말이 뛰고 있다


In [0]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [0]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


# RNN한계

 RNN은 출력 결과가 이전의 계산 결과에 의존한다는 것을 언급한 바 있습니다. 하지만, 앞서 배운 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점이 있습니다. 즉, RNN의 시점(time-step)이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생

# LSTM

RNN의  단점을 보완한 RNN의 일종을 장단기 메모리(Long Short-Term Memory)라고 하며, 줄여서 LSTM이라고 함
LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정함

# LSTM을 이용하여 텍스트 생성
데이터: 뉴욕 타임즈 기사의 제목

In [67]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/ArticlesApril2018.csv',encoding='utf-8')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [68]:
print(df.columns)
print('열의 개수: ',len(df.columns))

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
열의 개수:  15


In [69]:
df['headline'].isnull().values.any()

False

In [70]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [71]:
len(headline) # 현재 샘플의 개수

1324

In [72]:
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
len(headline) # 제거 후 샘플의 개수

1214

In [73]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

데이터 전처리

In [74]:
from string import punctuation
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

단어 집합 만들기

In [76]:
from keras_preprocessing.text import Tokenizer

t=Tokenizer()
t.fit_on_texts(text)
vocab_size=len(t.word_index)+1
print('단업 집합의 크기: %d ' % vocab_size)

단업 집합의 크기: 3494 


LSTM 으로 문장생성 하기

In [77]:
sequences= list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만듬

In [78]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key            #인덱스= 단어

index_to_word[582]

'offer'

패딩 작업을 수행하기 전에 가장 긴 샘플의 길이를 확인함



In [79]:


max_len=max(len(l) for l in sequences)
print(max_len)

24


모든 샘플의 길이를 24로 맞춤

In [81]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


x데이터와 y데이터 분리

In [0]:
import numpy as np

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [85]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [86]:
print(y[:3]) # 레이블

[ 269  371 1115]


y 데이터에 대한 원핫 인코딩

In [0]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [93]:

len(y[2])  # 단어 수

len(y)   # 샘플 수



7803

모델 설계

In [94]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
 - 10s - loss: 7.6502 - acc: 0.0309
Epoch 2/200
 - 9s - loss: 7.1222 - acc: 0.0301
Epoch 3/200
 - 9s - loss: 6.9849 - acc: 0.0363
Epoch 4/200
 - 9s - loss: 6.8623 - acc: 0.0423
Epoch 5/200
 - 9s - loss: 6.7206 - acc: 0.0427
Epoch 6/200
 - 9s - loss: 6.5645 - acc: 0.0459
Epoch 7/200
 - 9s - loss: 6.3880 - acc: 0.0506
Epoch 8/200
 - 9s - loss: 6.2036 - acc: 0.0536
Epoch 9/200
 - 9s - loss: 6.0223 - acc: 0.0596
Epoch 10/200
 - 9s - loss: 5.8451 - acc: 0.0637
Epoch 11/200
 - 9s - loss: 5.6704 - acc: 0.0677
Epoch 12/200
 - 9s - loss: 5.4978 - acc: 0.0724
Epoch 13/200
 - 9s - loss: 5.3295 - acc: 0.0779
Epoch 14/200
 - 9s - loss: 5.1673 - acc: 0.0843
Epoch 15/200
 - 9s - loss: 5.0144 - acc: 0.0920
Epoch 16/200
 - 9s - loss: 4.8664 - acc: 0.1035
Epoch 17/200
 - 9s - loss: 4.7265 - acc: 0.1139
Epoch 18/200
 - 9s - loss: 4.5915 - acc: 0.1266
Epoch 19/200
 - 9s - loss: 4.4606 - acc: 0.1453
Epoch 20/200
 - 9s - loss: 4.3354 - acc: 0.1601
Epoch 21/200
 - 9s - loss: 4.2156 - acc: 0.1802


LSTM 으로 문장 생성

In [0]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

확인

In [96]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i cant jump ship from facebook yet help life hes president


In [97]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

how to prevent a racist hoodie of ryan was in a
